In [ ]:
import sys; sys.path.insert(0, '../../')

In [ ]:
from pathlib import Path
from models.snn.factory import build_snn
from models.snn.train import train_model, evaluate_model
from models.snn.tuning import tune_hyperparameters
from models.preprocessing import load_preprocessed

In [ ]:
splits, class_names = load_preprocessed('../../data/processed/')
X_train, y_train = splits['train']
X_val, y_val = splits['val']
X_test, y_test = splits['test']

In [ ]:
checkpoint_dir = Path('../../checkpoints/snn')
env_path = str((checkpoint_dir / "best_hparams.env").absolute())

In [ ]:
best_config, analysis = tune_hyperparameters(
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    n_features=X_train.shape[1],
    n_classes=len(class_names),
    max_epochs=5,
    num_samples=20,
    project_name="snn_ray_tune",
 )

In [ ]:
print("\nBest hyperparameters found by Ray Tune:\n")
print(f"  Hidden neurons: {best_config['n_neurons_hidden']}")
print(f"  Synapse fast: {best_config.get('synapse_fast')}")
print(f"  Synapse slow: {best_config.get('synapse_slow')}")
print(f"  Learning rate: {best_config['learning_rate']:.2e}")
print(f"  Batch size: {best_config['batch_size']}")

best_hidden = int(best_config['n_neurons_hidden'])
best_syn_fast = float(best_config.get('synapse_fast'))
best_syn_slow = float(best_config.get('synapse_slow'))
best_lr = float(best_config['learning_rate'])
best_batch_sz = int(best_config['batch_size'])

In [ ]:
print(f"\nSaving best hyperparameters to {env_path} ...")
with open(env_path, "w") as f:
    f.write(f"N_NEURONS_HIDDEN={best_hidden}\n")
    f.write(f"SYNAPSE_FAST={best_syn_fast}\n")
    f.write(f"SYNAPSE_SLOW={best_syn_slow}\n")
    f.write(f"LEARNING_RATE={best_lr}\n")
    f.write(f"BATCH_SIZE={best_batch_sz}\n")
    f.write(f"N_FEATURES={X_train.shape[1]}\n")
    f.write(f"N_CLASSES={len(class_names)}\n")
print("Best hyperparameters saved.\n")